Created by Joan-Marc Fisa

- Numerai: [FisaGol](https://numer.ai/fisagol)

- Twitter: [@fisagol](https://twitter.com/fisagol)


In [1]:

from google.colab import drive
drive.mount('drive')

Mounted at drive


In [2]:
!pip install numerapi
!pip install vecstack;

  Created wheel for vecstack: filename=vecstack-0.4.0-cp37-none-any.whl size=19877 sha256=1d5c8e5248e025b9cf16ac1b0d57ccfc0ba8614d0645dd9e11270121880392d8
  Stored in directory: /root/.cache/pip/wheels/5f/bb/4e/f6488433d53bc0684673d6845e5bf11a25240577c8151c140e
Successfully built vecstack


In [3]:
##################################################################
##################### LIBRARIES ##################################
##################################################################


In [4]:
import os
import gc
import csv
import sys
import glob
import time
from pathlib import Path
from multiprocessing import Pool

import numerapi

import scipy
import numpy as np
import pandas as pd
import tensorflow as tf
import random
import sklearn
from sklearn import (
    feature_extraction, feature_selection, decomposition, linear_model,
    model_selection, metrics, svm, preprocessing, utils
)
from sklearn.preprocessing import StandardScaler,MinMaxScaler, OrdinalEncoder, LabelEncoder,OneHotEncoder
from keras.models import Sequential, model_from_json, load_model
from keras.layers import Dense, Dropout, Activation,LSTM,Bidirectional, MaxPooling2D, Flatten,GRU
from keras.optimizers import SGD,Adam
from keras.regularizers import l2
from sklearn.model_selection import StratifiedKFold, GroupKFold, GridSearchCV,cross_val_score,KFold, RepeatedStratifiedKFold,train_test_split
from sklearn.metrics import log_loss, make_scorer, mean_squared_error,classification_report,accuracy_score
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils 
from sklearn import preprocessing
from xgboost import XGBRegressor 
from sklearn.cluster import KMeans
import matplotlib as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler,MinMaxScaler

import math
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, BaggingRegressor
from sklearn.linear_model import SGDRegressor
from lightgbm import LGBMRegressor
from sklearn.tree import DecisionTreeRegressor 

import torch.nn as nn
import torch.nn.functional as F
import torch
from vecstack import stacking

from sklearn import metrics

def RMSLE(y, pred):
    return metrics.mean_squared_error(y, pred) ** 0.5


In [5]:
##############################################################################
########################## DOWLOAD DATA ######################################
##############################################################################

In [6]:
TOURNAMENT_NAME = "nomi"
TARGET_NAME = f"target"
PREDICTION_NAME = f"prediction"

BENCHMARK = 0
BAND = 0.2

In [7]:
seed_1 = 1
rand_1 = np.random.seed(seed_1)
seed_2 = rand_1
rand_2 = np.random.seed(seed_2)
seed_3 = rand_2
rand_3 = np.random.seed(seed_3)

In [8]:
napi = numerapi.NumerAPI(verbosity="info")

napi.download_current_dataset(unzip=True)

current_ds = napi.get_current_round()
latest_round = os.path.join('numerai_dataset_'+str(current_ds))

./numerai_dataset_258.zip: 398MB [00:08, 49.3MB/s]                           
2021-04-03 18:14:19,339 INFO numerapi.base_api: unzipping file...


In [9]:
##################################################################
##################### LOAD DATA ##################################
##################################################################

In [10]:
%%time
print("# Loading data...")

training_data = pd.read_csv(os.path.join(latest_round, "numerai_training_data.csv")).set_index("id")
tournament_data = pd.read_csv(os.path.join(latest_round, "numerai_tournament_data.csv")).set_index("id")
validation_data = tournament_data[tournament_data.data_type == "validation"]

print("# All Loaded...")

# Loading data...
# All Loaded...
CPU times: user 1min 6s, sys: 7.95 s, total: 1min 14s
Wall time: 1min 15s


In [11]:
feature_names = [f for f in training_data.columns if f.startswith("feature")]
print(f"Loaded {len(feature_names)} features")

Loaded 310 features


In [12]:
X = training_data[feature_names]
Y = training_data[training_data.columns[-1]]

le = LabelEncoder()
Y_enc = le.fit_transform(Y)

In [13]:
#############################################################################
###########################  CREATING SOME MODELS  ##########################
#############################################################################

In [14]:
%%time

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=rand_2)

model_x = [
          
    LGBMRegressor(random_state=rand_1),
    LGBMRegressor(random_state=rand_2),
    LGBMRegressor(random_state=rand_3)
]

from sklearn.metrics import explained_variance_score

S_train, S_test = stacking(model_x,                   
                          X_train, y_train, X_test,   
                          regression=True, 
                          mode='oof_pred_bag',
                          needs_proba=False,
                          save_dir=None,
                          metric= explained_variance_score,
                          n_folds=7,
                          stratified=True,
                          shuffle=True,
                          random_state=rand_3,verbose=2)


del model_x

gc.collect()

/usr/local/lib/python3.7/dist-packages/vecstack/core.py:461: UserWarning: This is regression task hence classification-specific parameters set to <True> were ignored: <stratified>
  warnings.warn(warn_str, UserWarning)


task:         [regression]
metric:       [explained_variance_score]
mode:         [oof_pred_bag]
n_models:     [3]

model  0:     [LGBMRegressor]
    fold  0:  [0.00193251]
    fold  1:  [0.00123256]
    fold  2:  [0.00201467]
    fold  3:  [0.00166922]
    fold  4:  [0.00174768]
    fold  5:  [0.00097359]
    fold  6:  [0.00165007]
    ----
    MEAN:     [0.00160290] + [0.00034619]
    FULL:     [0.00159456]

model  1:     [LGBMRegressor]
    fold  0:  [0.00172788]
    fold  1:  [0.00130297]
    fold  2:  [0.00122940]
    fold  3:  [0.00159287]
    fold  4:  [0.00125159]
    fold  5:  [0.00117073]
    fold  6:  [0.00091494]
    ----
    MEAN:     [0.00131291] + [0.00025071]
    FULL:     [0.00130998]

model  2:     [LGBMRegressor]
    fold  0:  [0.00161464]
    fold  1:  [0.00124304]
    fold  2:  [0.00169512]
    fold  3:  [0.00133981]
    fold  4:  [0.00125050]
    fold  5:  [0.00242861]
    fold  6:  [0.00119135]
    ----
    MEAN:     [0.00153758] + [0.00040582]
    FULL:     [0.0

In [15]:
model_e = LGBMRegressor(random_state=rand_1)
gc.collect()

50

In [16]:
model_1 = LGBMRegressor(random_state=rand_1)
model_1.fit(training_data[feature_names], training_data[TARGET_NAME])
gc.collect()

32

In [17]:
model_2 = LGBMRegressor(random_state=rand_2)
model_2.fit(training_data[feature_names], training_data[TARGET_NAME])
gc.collect()

31

In [18]:
model_3 = LGBMRegressor(random_state=rand_3)
model_3.fit(training_data[feature_names], training_data[TARGET_NAME])
gc.collect()

31

In [19]:
y_pred_model_1 = model_1.predict(tournament_data[feature_names])
gc.collect()

54

In [20]:
y_pred_model_2 = model_2.predict(tournament_data[feature_names])
gc.collect()

54

In [21]:
y_pred_model_3 = model_3.predict(tournament_data[feature_names])
gc.collect()

54

In [22]:
df_mix = pd.DataFrame(columns=['f0','f1','f2'])
df_mix['f0'] = y_pred_model_1
df_mix['f1'] = y_pred_model_2
df_mix['f2'] = y_pred_model_3

In [23]:
df_mix

,f0,f1,f2
0,0.488480,0.488480,0.488480
1,0.487922,0.487922,0.487922
2,0.516811,0.516811,0.516811
3,0.499933,0.499933,0.499933
4,0.502267,0.502267,0.502267
...,...,...,...
1687611,0.485124,0.485124,0.485124
1687612,0.434347,0.434347,0.434347
1687613,0.504729,0.504729,0.504729
1687614,0.501692,0.501692,0.501692


In [24]:
model_e = model_e.fit(S_train, y_train)
y_pred = model_e.predict(S_test)
gc.collect()

95

In [25]:
y_pred_final = model_e.predict(df_mix)
tournament_data[PREDICTION_NAME] = y_pred_final
gc.collect()

54

In [26]:
tournament_data[PREDICTION_NAME]

id
n0003aa52cab36c2    0.493578
n000920ed083903f    0.490939
n0038e640522c4a6    0.520027
n004ac94a87dc54b    0.499916
n0052fe97ea0c05f    0.503135
                      ...   
nffd0e068bc831f1    0.485977
nffd81c58a66f813    0.456503
nfff50e539a0e581    0.507352
nfff8f00d3537014    0.501271
nfffae0a5193a0c9    0.499022
Name: prediction, Length: 1687616, dtype: float64

In [27]:
import numerapi
import torch
from torch.nn import Linear
from torch.nn import Sequential
from torch.functional import F

In [28]:
def exposures(x, y):
    x = x - x.mean(dim=0)
    x = x / x.norm(dim=0)
    y = y - y.mean(dim=0)
    y = y / y.norm(dim=0)
    return torch.matmul(x.T, y)

def reduce_exposure(prediction, features, max_exp):
    # linear model of features that will be used to partially neutralize predictions
    lin = Linear(features.shape[1],  1, bias=False)
    lin.weight.data.fill_(0.)
    model = Sequential(lin)
    optimizer = torch.optim.Adamax(model.parameters(), lr=1e-4)
    feats = torch.tensor(np.float32(features)-.5)
    pred = torch.tensor(np.float32(prediction))
    start_exp = exposures(feats, pred[:,None])
    # set target exposure for each feature to be <= current exposure
    # if current exposure is less than max_exp, or <= max_exp if  
    # current exposure is > max_exp
    targ_exp = torch.clamp(start_exp, -max_exp, max_exp)

    for i in range(100000):
        optimizer.zero_grad()
        # calculate feature exposures of current linear neutralization
        exps = exposures(feats, pred[:,None]-model(feats))
        # loss is positive when any exposures exceed their target
        loss = (F.relu(F.relu(exps)-F.relu(targ_exp)) + F.relu(F.relu(-exps)-F.relu(-targ_exp))).sum()
        print(f'       loss: {loss:0.7f}', end='\r')
        if loss < 1e-7:
            neutralizer = [p.detach().numpy() for p in model.parameters()]
            neutralized_pred = pred[:,None]-model(feats)
            break
        loss.backward()
        optimizer.step()
    return neutralized_pred, neutralizer

def reduce_all_exposures(df, column, neutralizers=[],
                                     normalize=True,
                                     gaussianize=True,
                                     era_col="era",
                                     max_exp=0.1):
    unique_eras = df[era_col].unique()
    computed = []
    for u in unique_eras:
        print(u, '\r')
        df_era = df[df[era_col] == u]
        scores = df_era[column].values
        exposure_values = df_era[neutralizers].values
        
        if normalize:
            scores2 = []
            for x in scores.T:
                x = (scipy.stats.rankdata(x, method='ordinal') - .5) / len(x)
                if gaussianize:
                    x = scipy.stats.norm.ppf(x)
                scores2.append(x)
            scores = np.array(scores2)[0]

        scores, neut = reduce_exposure(scores, exposure_values, max_exp)

        scores /= scores.std()

        computed.append(scores.detach().numpy())

    return pd.DataFrame(np.concatenate(computed), columns=column, index=df.index)

In [29]:
tournament_data[PREDICTION_NAME]

id
n0003aa52cab36c2    0.493578
n000920ed083903f    0.490939
n0038e640522c4a6    0.520027
n004ac94a87dc54b    0.499916
n0052fe97ea0c05f    0.503135
                      ...   
nffd0e068bc831f1    0.485977
nffd81c58a66f813    0.456503
nfff50e539a0e581    0.507352
nfff8f00d3537014    0.501271
nfffae0a5193a0c9    0.499022
Name: prediction, Length: 1687616, dtype: float64

In [30]:
data_rfe_10 = reduce_all_exposures(tournament_data,
                                   [PREDICTION_NAME],
                                   neutralizers=feature_names,
                                   era_col="era",
                                   max_exp=0.05)

era121 
era122 
era123 
era124 
era125 
era126 
era127 
era128 
era129 
era130 
era131 
era132 
era575 
era576 
era577 
era578 
era579 
era580 
era581 
era582 
era583 
era584 
era585 
era586 
era587 
era588 
era589 
era590 
era591 
era592 
era593 
era594 
era595 
era596 
era597 
era598 
era599 
era600 
era601 
era602 
era603 
era604 
era605 
era606 
era607 
era608 
era609 
era610 
era611 
era612 
era613 
era614 
era615 
era616 
era617 
era618 
era619 
era620 
era621 
era622 
era623 
era624 
era625 
era626 
era627 
era628 
era629 
era630 
era631 
era632 
era633 
era634 
era635 
era636 
era637 
era638 
era639 
era640 
era641 
era642 
era643 
era644 
era645 
era646 
era647 
era648 
era649 
era650 
era651 
era652 
era653 
era654 
era655 
era656 
era657 
era658 
era659 
era660 
era661 
era662 
era663 
era664 
era665 
era666 
era667 
era668 
era669 
era670 
era671 
era672 
era673 
era674 
era675 
era676 
era677 
era678 
era679 
era680 
era681 
era682 
era683 
era684 
era685 
era686 
era687 


In [31]:
# replace prediction with reduced feature exposure prediction and rescale to [0,1]
tournament_data[PREDICTION_NAME] = data_rfe_10[PREDICTION_NAME]
tournament_data[PREDICTION_NAME] -= tournament_data[PREDICTION_NAME].min()
tournament_data[PREDICTION_NAME] /= tournament_data[PREDICTION_NAME].max()

2021-04-03 21:28:49,640 INFO numexpr.utils: Note: NumExpr detected 40 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2021-04-03 21:28:49,641 INFO numexpr.utils: NumExpr defaulting to 8 threads.


In [32]:
tournament_data[PREDICTION_NAME]

id
n0003aa52cab36c2    0.528519
n000920ed083903f    0.466151
n0038e640522c4a6    0.489842
n004ac94a87dc54b    0.532984
n0052fe97ea0c05f    0.556546
                      ...   
nffd0e068bc831f1    0.323237
nffd81c58a66f813    0.302718
nfff50e539a0e581    0.511918
nfff8f00d3537014    0.533662
nfffae0a5193a0c9    0.459555
Name: prediction, Length: 1687616, dtype: float64

In [33]:
##############################################################################
######################### MAKE SUBMISSION #################################### 
##############################################################################

In [34]:
tournament_data[PREDICTION_NAME].to_csv(f"{TOURNAMENT_NAME}_{current_ds}_submission.csv")

In [35]:
# NameOfYourAI FISAGOL: jmfisagol@gmail.com
public_id = "7TISUDGAWEVO2B35ECOQQXU2RWXGZN3I"
secret_key = "QJYUWIMFEEDNZ4GHUO6VSSKPMRCBFJIMJ7BZ65ESIWRN4YHGYHSRJDNL64TAG7EH"
model_id = "d49c26a4-aa5b-4490-9d58-300c5e05d996"
napi = numerapi.NumerAPI(public_id=public_id, secret_key=secret_key)

In [36]:
print("Uploading DataFrame in Numerai...")
submission_id = napi.upload_predictions(f"{TOURNAMENT_NAME}_{current_ds}_submission.csv", model_id=model_id)
print("DataFrame Uploaded...")

2021-04-03 21:28:53,521 INFO numerapi.base_api: uploading predictions...


Uploading DataFrame in Numerai...
DataFrame Uploaded...


In [37]:
tournament_data[PREDICTION_NAME].to_csv('FisaGol1_1.csv')
!cp FisaGol1_1.csv "drive/My Drive/PHOENIXSIGMA/"

In [38]:
'''
##########################################################################################
##############################   SAVE MODEL   ############################################
##########################################################################################
from sklearn.externals import joblib
joblib.dump(my_model, "my_model.pkl")
# and later...
my_model_loaded = joblib.load("my_model.pkl")
'''

'\n##########################################################################################\n##############################   SAVE MODEL   ############################################\n##########################################################################################\nfrom sklearn.externals import joblib\njoblib.dump(my_model, "my_model.pkl")\n# and later...\nmy_model_loaded = joblib.load("my_model.pkl")\n'